In [1]:
import pandas as pd

In [2]:
all_cols_df = pd.read_csv('game_data/game_stats.csv')

In [3]:
import numpy as np
df = all_cols_df.iloc[:, 0:12]
df['Result'] = np.where(df['HomeScore'] > df['AwayScore'], 1, -1)


df.head()

,Date,KickOff,HomeTeam,AwayTeam,Venue,HomeScore,AwayScore,PlayOff Game?,HomeGoals,HomeBehinds,AwayGoals,AwayBehinds,Result
0,25-Apr-18,15:20,Collingwood,Essendon,MCG,101,52,NaN,14,17,7,10,1
1,24-Apr-18,19:25,Melbourne,Richmond,MCG,56,102,NaN,8,8,15,12,-1
2,22-Apr-18,16:40,Brisbane,Gold Coast,Gabba,71,76,NaN,10,11,11,10,-1
3,22-Apr-18,15:20,North Melbourne,Hawthorn,Etihad Stadium,98,70,NaN,14,14,11,4,1
4,21-Apr-18,18:10,Fremantle,Western Bulldogs,Simonds Stadium,108,54,NaN,16,12,8,6,1


In [4]:
import datetime as dt
teams = ['Collingwood', 'Essendon', 'Brisbane', 'Gold Coast', 'North Melbourne', 'Hawthorn', 'Fremantle', 'Western Bulldogs', 'Port Adelaide', 'Geelong', 'Carlton', 'West Coast', 'St Kilda', 'GWS Giants', 'Sydney', 'Adelaide', 'Melbourne', 'Richmond']

datetimes = []
dates, times = df['Date'], df['KickOff']

for i in range(len(dates)):
    date_str = str(dates[i]) + " " + str(times[i]) + ":00"
    date = dt.datetime.strptime(date_str, "%d-%b-%y %H:%M:%S")
    
    datetimes.append(date)

print(datetimes[-1])

df['Datetime'] = datetimes

df = df.iloc[:, range(2,14)]


df = df.set_index('Datetime')

df_17, df_18, df_16 = df['2017'], df['2018'], df['2016']





2009-06-19 19:40:00


In [5]:
len(df_18['AwayScore'])

45

In [6]:
scores_for, scores_against, num_games = dict(), dict(), dict()

for team in teams:
    scores_for[team], scores_against[team] = 0, 0    
    num_games[team] = 0
    
for home_team, away_team, home, away in zip(df_17['HomeTeam'], df_17['AwayTeam'], df_17['HomeScore'], df_17['AwayScore']):
    scores_for[home_team] += home
    scores_for[away_team] += away
    scores_against[home_team] += away
    scores_against[away_team] += home

    num_games[home_team] += 1
    num_games[away_team] += 1
    
    
    

In [7]:


team_df = pd.DataFrame()

team_df['GamesPlayed'] = pd.Series(num_games)
team_df['For'] = pd.Series(scores_for)
team_df['Against'] = pd.Series(scores_against)


team_df.head()

,GamesPlayed,For,Against
Adelaide,25,2695,2007
Brisbane,22,1877,2526
Carlton,22,1594,2038
Collingwood,22,1944,1963
Essendon,23,2191,2125


In [8]:
avgs_for, avgs_against = dict(), dict()
for team in teams:
    avgs_for[team], avgs_against[team] = 0,0
    
for team, games, F, A in zip(team_df.index, team_df['GamesPlayed'], team_df['For'], team_df['Against']) :
    avgs_for[team] = F/games
    avgs_against[team] = A/games

team_df['AvgFor'] = pd.Series(avgs_for)
team_df['AvgAgainst'] = pd.Series(avgs_against)


In [9]:
team_df

,GamesPlayed,For,Against,AvgFor,AvgAgainst
Adelaide,25,2695,2007,107.800000,80.280000
Brisbane,22,1877,2526,85.318182,114.818182
Carlton,22,1594,2038,72.454545,92.636364
Collingwood,22,1944,1963,88.363636,89.227273
Essendon,23,2191,2125,95.260870,92.391304
Fremantle,22,1607,2160,73.045455,98.181818
GWS Giants,25,2321,2057,92.840000,82.280000
Geelong,25,2347,2084,93.880000,83.360000
Gold Coast,22,1756,2311,79.818182,105.045455
Hawthorn,22,1864,2055,84.727273,93.409091


In [10]:
t_score, t_games = 0, 0

for team, games, F, A in zip(team_df.index, team_df['GamesPlayed'], team_df['For'], team_df['Against']):
    t_score += F
    t_games += games
                 
avg_score = t_score/t_games



In [11]:
attacks = dict()
defences = dict()

for team, avg_for, avg_against in zip(team_df.index, team_df['AvgFor'], team_df['AvgAgainst']):
    attacks[team] = avg_for/avg_score
    defences[team] = avg_against/avg_score

team_df['Attack'] = pd.Series(attacks)
team_df['Defence'] = pd.Series(defences)


team_df

,GamesPlayed,For,Against,AvgFor,AvgAgainst,Attack,Defence
Adelaide,25,2695,2007,107.800000,80.280000,1.209890,0.901020
Brisbane,22,1877,2526,85.318182,114.818182,0.957566,1.288658
Carlton,22,1594,2038,72.454545,92.636364,0.813191,1.039701
Collingwood,22,1944,1963,88.363636,89.227273,0.991746,1.001439
Essendon,23,2191,2125,95.260870,92.391304,1.069157,1.036951
Fremantle,22,1607,2160,73.045455,98.181818,0.819823,1.101940
GWS Giants,25,2321,2057,92.840000,82.280000,1.041987,0.923467
Geelong,25,2347,2084,93.880000,83.360000,1.053659,0.935588
Gold Coast,22,1756,2311,79.818182,105.045455,0.895837,1.178974
Hawthorn,22,1864,2055,84.727273,93.409091,0.950934,1.048374


In [12]:
r6 = [['Western Bulldogs', 'Carlton'], ['Geelong', 'Sydney'], ['North Melbourne','Port Adelaide'], 
      ['GWS Giants', 'Brisbane'], ['Hawthorn', 'St Kilda'], ['Adelaide', 'Gold Coast'], ['Essendon', 'Melbourne'], ['Collingwood', 'Richmond'], ['Fremantle', 'West Coast']]
for r in r6:
    home_score = (team_df.iloc[teams.index(r[0]),5]) * (team_df.iloc[teams.index(r[1]), 6]) * avg_score
    away_score = (team_df.iloc[teams.index(r[1]),5] )* (team_df.iloc[teams.index(r[0]), 6]) * avg_score
    print(r[0] ,home_score, r[1],away_score)

Western Bulldogs 92.62620593113611 Carlton 86.54190365169302
Geelong 85.84337737297761 Sydney 91.73270578203208
North Melbourne 112.31009939446517 Port Adelaide 82.76751849013078
GWS Giants 95.40297310952124 Brisbane 60.2086724813127
Hawthorn 62.34220728169816 St Kilda 107.51104725241956
Adelaide 94.01011288776289 Gold Coast 74.58758567319848
Essendon 82.15116036199007 Melbourne 112.75757411455623
Collingwood 105.20540569848455 Richmond 76.05426824429401
Fremantle 107.22989454279285 West Coast 83.23793965353647
